In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datasets import load_dataset
import seaborn as sns
import ast
import math

import scipy
import openpyxl
import requests

from matplotlib.ticker import FuncFormatter

In [2]:
df = pd.read_csv(
    r"C:\Users\Administrator\Documents\CSV Files\global_house_purchase_dataset.csv"
)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 25 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   property_id              200000 non-null  int64  
 1   country                  200000 non-null  object 
 2   city                     200000 non-null  object 
 3   property_type            200000 non-null  object 
 4   furnishing_status        200000 non-null  object 
 5   property_size_sqft       200000 non-null  int64  
 6   price                    200000 non-null  int64  
 7   constructed_year         200000 non-null  int64  
 8   previous_owners          200000 non-null  int64  
 9   rooms                    200000 non-null  int64  
 10  bathrooms                200000 non-null  int64  
 11  garage                   200000 non-null  int64  
 12  garden                   200000 non-null  int64  
 13  crime_cases_reported     200000 non-null  int64  
 14  lega

In [4]:
df_copy = df.copy()

In [5]:
df_copy

,property_id,country,city,property_type,furnishing_status,property_size_sqft,price,constructed_year,previous_owners,rooms,...,customer_salary,loan_amount,loan_tenure_years,monthly_expenses,down_payment,emi_to_income_ratio,satisfaction_score,neighbourhood_rating,connectivity_score,decision
0,1,France,Marseille,Farmhouse,Semi-Furnished,991,412935,1989,6,6,...,10745,193949,15,6545,218986,0.16,1,5,6,0
1,2,South Africa,Cape Town,Apartment,Semi-Furnished,1244,224538,1990,4,8,...,16970,181465,20,8605,43073,0.08,9,1,2,0
2,3,South Africa,Johannesburg,Farmhouse,Semi-Furnished,4152,745104,2019,5,2,...,21914,307953,30,2510,437151,0.09,6,8,1,0
3,4,Germany,Frankfurt,Farmhouse,Semi-Furnished,3714,1110959,2008,1,3,...,17980,674720,15,8805,436239,0.33,2,6,6,0
4,5,South Africa,Johannesburg,Townhouse,Fully-Furnished,531,99041,2007,6,3,...,17676,65833,25,8965,33208,0.03,3,3,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,199996,Germany,Berlin,Villa,Fully-Furnished,685,203328,1968,1,3,...,78330,104050,15,17670,99278,0.01,8,4,5,1
199996,199997,China,Shenzhen,Townhouse,Unfurnished,3818,1454627,1977,5,7,...,25400,1175297,20,2865,279330,0.34,7,10,9,1
199997,199998,Japan,Kyoto,Villa,Semi-Furnished,3603,1619147,1990,2,4,...,28220,743049,30,5595,876098,0.17,5,3,9,0
199998,199999,South Africa,Johannesburg,Apartment,Unfurnished,1706,306165,2010,0,4,...,12240,150774,15,16300,155391,0.11,6,10,6,0


In [ ]:
# Example
df_ex = df_copy.copy()

In [ ]:
# Example
df_ex['pct_sales'] = df_copy['price'] / df_copy.groupby('property_type')['price'].transform('sum')

df_ex

In [ ]:
df_ex.groupby('property_type')['price'].agg(['mean', 'median', 'std']).reset_index()

In [6]:
# Problem 1: Market Affordability by Country
# Which countries offer the most affordable properties when price is normalized by property size and customer income?
# Step 3: Create derived metrics (core learning step)
# Price per Square Foot

df_copy["price_per_sqft"] = df_copy["price"] / df_copy["property_size_sqft"]

# Column-wise operations
# Vectorized arithmetic (no loops)

In [7]:
df_copy['price_per_sqft']

0         416.685166
1         180.496785
2         179.456647
3         299.127356
4         186.517891
             ...    
199995    296.829197
199996    380.991881
199997    449.388565
199998    179.463658
199999    200.629244
Name: price_per_sqft, Length: 200000, dtype: float64

In [8]:
# 3.2 Price-to-Income Ratio

df_copy["price_to_income_ratio"] = df_copy["price"] / df_copy["customer_salary"]

# Key concept: Pandas handles row-level math automatically.

In [14]:
# Problem 1: Market Affordability by Country
# Which countries offer the most affordable properties when price is normalized by property size and customer income?
# Step 4: Aggregate by Country
# Why: Reporting = grouped summaries

country_affordability = (
    df_copy.groupby("country")
    .agg(
        avg_price_per_sqft=("price_per_sqft", "mean"),
        avg_price_to_income=("price_to_income_ratio", "mean"),
        avg_salary=("customer_salary", "mean"),
    )
    .reset_index()
)

# Named aggregations
# Clean output formatting

In [ ]:
country_affordability

,country,avg_price_per_sqft,avg_price_to_income,avg_salary
0,Australia,320.005783,26.029335,55246.344062
1,Brazil,200.006833,41.384339,21937.720205
2,Canada,350.010676,28.216718,55182.932926
3,China,379.992591,31.185831,54905.907891
4,France,420.019822,34.351656,54988.787113
5,Germany,299.984573,24.608471,55039.591121
6,India,150.009025,60.488816,11016.191639
7,Japan,449.996153,37.056781,54613.850950
8,Singapore,699.961154,57.719955,54877.393638
9,South Africa,180.010247,36.731516,21968.630479


In [17]:
# Step 5: Sort for Insights

country_affordability.sort_values(by="avg_price_to_income")
# This produces your affordability ranking.

,country,avg_price_per_sqft,avg_price_to_income,avg_salary
5,Germany,299.984573,24.608471,55039.591121
0,Australia,320.005783,26.029335,55246.344062
2,Canada,350.010676,28.216718,55182.932926
3,China,379.992591,31.185831,54905.907891
11,UK,399.983922,33.044174,55104.828716
4,France,420.019822,34.351656,54988.787113
9,South Africa,180.010247,36.731516,21968.630479
7,Japan,449.996153,37.056781,54613.850950
12,USA,499.985016,41.384286,54950.747333
1,Brazil,200.006833,41.384339,21937.720205


In [18]:
# Step 6: Sanity Check (Professionatl Habit)

country_affordability.describe()
# Ensures no extreme or illogical values

,avg_price_per_sqft,avg_price_to_income,avg_salary
count,13.000000,13.000000,13.000000
mean,380.765788,38.535093,46527.087702
std,160.273232,11.308583,16293.584072
min,150.009025,24.608471,11016.191639
25%,299.984573,31.185831,54613.850950
50%,379.992591,36.731516,54950.747333
75%,449.996153,41.384339,55039.591121
max,699.961154,60.488816,55246.344062


In [ ]:
# Step 7: Interpretation (Mandatory)
# “Countries with lower average price-to-income ratios indicate stronger affordability, even when absolute prices vary.”

In [ ]:
# Problem 1 (Extended): Affordability by Property Size Category
# Refined analytical question: Within each country, how does affordability differ across property size categories?
# This answers: Are smaller properties truly more affordable? Do some countries price larger properties more efficiently?

In [19]:
# Create property_size_category
# using Pandas approach: pd.cut() — which is the correct tool

bins = [0, 1000, 2500, 6000]
labels = ["Small", "Medium", "Large"]

df_copy["property_size_category"] = pd.cut(
    df_copy["property_size_sqft"], bins=bins, labels=labels, include_lowest=True
)

# Using pd.cut() is the correct tool
# It converts a continuous numeric variable into categorical segments
# it is vectorized (fast, clean)
# it preserves analytical intent
# it is widely used in real-world analytics
# this is exactly the right function for this task.

In [20]:
# Validate the new column (mandatory)
df_copy["property_size_category"].value_counts()

property_size_category
Large     124770
Medium     53887
Small      21343
Name: count, dtype: int64

In [ ]:
# Problem 1 (Extended): Affordability by Property Size Category
# Step 1: Group by Country and Size Category
size_affordability = (
    df_copy.groupby(["country", "property_size_category"], observed=True)
    .agg(
        avg_price_per_sqft=("price_per_sqft", "mean"),
        avg_price_to_income=("price_to_income_ratio", "mean"),
    )
    .reset_index()
)

In [ ]:
# Step 2: Compare Within Countries
# This reveals: Which size category is most affordable per country
size_affordability.sort_values(
    by=["country", "avg_price_to_income"], ascending=[True, True]
)

In [ ]:
# Step 3: Sanity Check
size_affordability.head(10)

# Confirm:
# Values make sense
# No unexpected spikes

In [ ]:
# Problem 2: Purchase Decision Drivers
# What financial and affordability factors are associated with a customer’s decision to purchase a property?

# Understand the Decision Variable
# Why: Always validate the target variable first.

df_copy["decision"].value_counts(normalize=True)

# what you learn: Overall purchase rate.

In [ ]:
# Step 2: Buyers vs Non-Buyers Comparison

decision_summary = df_copy.groupby("decision").agg(
    avg_salary=("customer_salary", "mean"),
    avg_price_to_income=("price_to_income_ratio", "mean"),
    avg_emi_ratio=("emi_to_income_ratio", "mean"),
    avg_down_payment=("down_payment", "mean"),
)


decision_summary

In [ ]:
# Step 3: Interpret the Differences (Critical Thinking)
# You are looking for:
# Lower ratios for buyers
# Higher salaries for buyers
# Higher down payments for buyers

# if you see these patterns, your analysis is behaving logically

In [ ]:
# Step 4: EMI-to-Income Segmentation (Important Learning Step)
# Why: Raw ratios are hard to interpret operationally

# Create EMI bands using pd.cut()

bins = [0, 0.30, 0.45, 1.0, df["emi_to_income_ratio"].max()]
labels = ["Low", "Medium", "High", "Extreme"]

df_copy["emi_band"] = pd.cut(
    df["emi_to_income_ratio"], bins=bins, labels=labels, include_lowest=True
)

In [ ]:
# Step 5: Purchase Rate by EMI Band

emi_conversion = (
    df_copy.groupby(["country", "emi_band"], observed=True)["decision"]
    .mean()
    .reset_index(name="purchase_rate")
)

# Important concept
# Since decision 0/1:
# Mean = purchase rate
# This is a key analytics trick

In [ ]:
emi_conversion

In [ ]:
size_conversion = (
    df_copy.groupby("property_size_category", observed=True)["decision"]
    .mean()
    .reset_index(name="purchase_rate")
)

size_conversion

In [ ]:
# Problem 3: Location Quality & Customer Satisfaction
# Step 1: Validate Relevant Columns
# Step 2: Aggregate City-level Metrics
# Why we do this:
# Operations reporting compares entities, not individuals.
# Cities must be summarized using averages.

city_satisfaction = (
    df_copy.groupby(["country", "city"])
    .agg(
        avg_satisfaction=("satisfaction_score", "mean"),
        avg_neighbourhood=("neighbourhood_rating", "mean"),
        avg_connectivity=("connectivity_score", "mean"),
        property_count=("property_id", "count"),
    )
    .reset_index()
)

In [ ]:
city_satisfaction

In [ ]:
# Step 3: Filter for Reliable Cities
# Why this matters
# Cities with very few listings can distort insights

city_satisfaction[city_satisfaction["property_count"] >= 20]

In [ ]:
# Use a Data-Driven Threshold (If You Must)
# If a stakeholder insists on a rule, you derive it from the data, not arbitrarily:


city_satisfaction["property_count"].describe()

# Then choose something like:
# Then choose something like:Bottom 5th percentile
# Or a clearly justified operational threshold
# But again: your dataset does not require this.

In [ ]:
# Step 4: Identify Top-Performing Cities (Required)
# Purpose
# Operations and reporting analysts are expected to:
# Rank entities
# Surface top and bottom performers
# Support prioritization

top_cities = city_satisfaction.sort_values(by="avg_satisfaction", ascending=False)

In [ ]:
top_cities

In [ ]:
# Step 5: Analyze Contribution Relationships
# Why this is important
# This is where the analysis moves from descriptive to diagnostic.
# We ask: When satisfaction is high, are neigbhourhood and connectivity also high?

city_satisfaction[["avg_satisfaction", "avg_neighbourhood", "avg_connectivity"]].corr()

# How an Analyst Interprets This
# You are not predicting, you are diagnosing:
# Moderate correlation → contributing factor
# Weak correlation → low influence
# This is diagnostic analytics, which is expected even in reporting roles.

In [ ]:
# Problem 4: Market Risk & Price Stability
# Business Framing
# Operational question: Which countries show higher price volatility, indicating higher market risk?
# This supports:
# Risk monitoring
# Market entry decisions
# Reporting stability metrics

# Step 1: Aggregate Country-Level Price Metrics

country_price_stats = (
    df_copy.groupby("country")
    .agg(
        avg_price=("price", "mean"),
        price_std=("price", "std"),
        min_price=("price", "min"),
        max_price=("price", "max"),
        property_count=("property_id", "count"),
    )
    .reset_index()
)

# Why we do this
# std = volatility
# min/max = spread context
# property_count = reliability

In [ ]:
# Step 2: Normalize Volatility (Critical Step)
# Raw standard deviation is misleading across price levels.

country_price_stats["price_volatility_ratio"] = (
    country_price_stats["price_std"] / country_price_stats["avg_price"]
)
# Why this matters
# This makes volatility comparable across countries
# This is professional-level reporting

In [ ]:
# Step 3: Filter for Data Reliability

country_price_stats = country_price_stats[country_price_stats["property_count"] >= 50]

In [ ]:
country_price_stats

In [ ]:
# Step 4: Rank Countries by Market Risk

price_risk_markets = country_price_stats.sort_values(
    by="price_volatility_ratio", ascending=False
)

price_risk_markets

In [ ]:
# Step 5: Interpret Market Risk
# How an analyst explains this:
# High volatility ratio → unstable pricing → higher risk
# Low volatility ratio → predictable market → operational stability

# This is not prediction, it is risk profiling, which is correct for a reporting analyst.

In [ ]:
# Problem 4: Risk & Market Stability Analysis

country_risk = (
    df_copy.groupby("country")
    .agg(
        avg_crime=("crime_cases_reported", "mean"),
        avg_legal_cases=("legal_cases_on_property", "mean"),
        purchase_rate=("decision", "mean"),
    )
    .reset_index()
)

In [ ]:
# Step 2: Normalize Risk Interpretation (High vs Low Risk)
# Why:
# Raw crime and legal counts are hard to interpret operationally.
# We need relative comparison, not raw numbers.
# Create Risk Bands (Optional but Strong)

country_risk["crime_risk_level"] = pd.qcut(
    country_risk["avg_crime"], q=3, labels=["Low", "Medium", "High"]
)

country_risk["legal_risk_level"] = pd.qcut(
    country_risk["avg_legal_cases"], q=3, labels=["Low", "Medium", "High"]
)

# Why qcut()
# Divides markets into equal-sized risk groups
# Common in reporting and policy analysis
# Avoids arbitrary thresholds

In [ ]:
# Step 3: Analyze Risk Impact on Purchase Decisions
# Key Question
# Do higher crime or legal risks correspond to lower purchase rates?

# Python Correlation Analysis

risk_correlation = df_copy[
    ["crime_cases_reported", "legal_cases_on_property", "decision"]
].corr()

risk_correlation

# How an Analyst Interprets This:
# Negative correlation with decision: → Risk discourages purchases
# Weak or no correlation: → Buyers may tolerate or ignore certain risks
# Stronger legal correlation than crime: → Legal clarity matters more than safety perception
# This is diagnostic analytics, not prediction.

In [ ]:
# Step 4: Identify High-Risk but High-Conversion Markets
# Why This Is Critical
# These markets:
# Look attractive on the surface
# Carry hidden operational or legal exposure
# Require policy, compliance, or mitigation strategies

operational_risk_markets = country_risk[
    (country_risk["crime_risk_level"] == "High")
    | (country_risk["legal_risk_level"] == "High")
].sort_values(by="purchase_rate", ascending=False)

operational_risk_markets

# Now:
# Both datasets coexist
# Each has a clear analytical meaning
# Your notebook reads like a report, not a script
# This is exactly how analysts should name variables.

In [ ]:
# Step 5: (Optional Drill-Down) City-Level Risk Analysis
# If needed for stronger portfolio depth:

city_risk = (
    df_copy.groupby(["country", "city"])
    .agg(
        avg_crime=("crime_cases_reported", "mean"),
        avg_legal=("legal_cases_on_property", "mean"),
        purchase_rate=("decision", "mean"),
        property_count=("property_id", "count"),
    )
    .reset_index()
)

city_risk

In [ ]:
# (Advanced but Valuable) Combined Risk View
# Once variables are clearly separated, you can do something very strong for a portfolio:
# Identify markets that are risky on multiple dimensions

combined_risk_markets = price_risk_markets[["country", "price_volatility_ratio"]].merge(
    operational_risk_markets[
        ["country", "avg_crime", "avg_legal_cases", "purchase_rate"]
    ],
    on="country",
    how="inner",
)

combined_risk_markets

# This allows statements like:
# These countries exhibit both high price volatility and elevated legal/crime risk, yet still maintain strong purchase conversion rates.

In [ ]:
# Defined a functions
def categorize_size(size):
    if size < 1500:
        return "Small"
    elif size < 3000:
        return "Medium"
    elif size < 4500:
        return "Large"
    else:
        return "Very Large"

In [ ]:
df_copy["property_size_category"] = df_copy["property_size_sqft"].apply(categorize_size)

df_copy["property_size_category"]

In [ ]:
df_copy.columns

In [ ]:
df_copy = df_copy[
    [
        "property_id",
        "country",
        "city",
        "property_type",
        "furnishing_status",
        "property_size_sqft",
        "property_size_category",
        "price",
        "constructed_year",
        "previous_owners",
        "rooms",
        "bathrooms",
        "garage",
        "garden",
        "crime_cases_reported",
        "legal_cases_on_property",
        "customer_salary",
        "loan_amount",
        "loan_tenure_years",
        "monthly_expenses",
        "down_payment",
        "emi_to_income_ratio",
        "satisfaction_score",
        "neighbourhood_rating",
        "connectivity_score",
        "decision",
    ]
]


df_copy

In [ ]:
df_copy

In [ ]:
df_copy["country"].unique()

In [ ]:
df_copy.groupby(["country", "property_type"])["property_type"].agg(
    ["size"]
).sort_values("size", ascending=False).head(40)

In [ ]:
df_copy.pivot_table(
    index="country", columns="property_type", aggfunc="size"
).sort_values("Apartment", ascending=False)

In [ ]:
df_property_type = df_pivoted.columns


def sort_countries(df_property_type):
    return df_pivoted.sort_values()

In [ ]:
df_pivoted.loc["total"] = df_pivoted.sum()

df_pivoted

In [ ]:
df_pivoted[df_pivoted.loc["total"].sort_values(ascending=False).index]

In [ ]:
df_pivoted = df_pivoted.drop("total")

df_pivoted

In [ ]:
df_pivoted

In [ ]:
df_pivoted.apply(lambda col: col.sort_values(ascending=False).reset_index(drop=False))

In [ ]:
df_pivoted

In [ ]:
col_index = df_copy.columns.get_loc("property_size_sqft")

df_copy.insert(
    loc=col_index + 1,
    column="property_size_category",
    value=df_copy.pop("property_size_category"),
)

df_copy

In [ ]:
dict = {
    "Time_of_Day": "time_of_day",
    "Weekday": "weekday",
    "Month_name": "month_name",
    "Weekdaysort": "weekday_sort",
    "Monthsort": "monthsort",
    "Date": "date",
    "Time": "time",
}


df_copy = df_copy.rename(columns=dict)

df_copy

In [ ]:
df_copy["coffee_name"].unique()

In [ ]:
df_groupby = df_copy.groupby(["country", "city"])["price"].agg(
    ["mean", "median", "min", "max"]
)

df_groupby

In [ ]:
# No need to reset the index.
numeric_cols = ["mean", "median", "min", "max"]

# By applying f"{}", it will turn the values to strings.
df_groupby = df_groupby[numeric_cols].round()

df_groupby

In [ ]:
df_groupby = df_groupby.reset_index()

df_groupby

In [ ]:
df_groupby = df_groupby.sort_values(by="median", ascending=False)

df_groupby

In [ ]:
df_groupby.head(10)

In [ ]:
df_groupby_top10

In [ ]:
df_groupby_top10.plot(x="city", y="median", kind="barh", figsize=(12, 8))
plt.ticklabel_format(style="plain", axis="x")  # <--- disables scientific notation
# Format the x-label with , on the numbers
plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, pos: f"{x:,.0f}"))
plt.title("Top 10 Cities by Median Home Price", fontsize=20)
plt.ylabel("City")
plt.xlabel("Median Price")
# plt.xlim(0, 2.500.000)
plt.gca().invert_yaxis()

plt.tight_layout()
plt.show()

In [ ]:
# We will create a column label for the decision
df_copy["decision_label"] = df_copy["decision"].map({0: "Denied", 1: "Approved"})

df_copy

In [ ]:
# Question 2 - What is the distribution of loan approvals vs denials across property types?

loan_distribution = (
    pd.crosstab(df_copy["property_type"], df_copy["decision_label"], normalize="index")
    * 100
)

loan_distribution

In [ ]:
loan_distribution = loan_distribution.round(2)

loan_distribution

In [ ]:
loan_distribution.plot(kind="barh", stacked=True, figsize=(12, 10))
plt.title("Loan Approval vs Denial Rate by Property Type", fontsize=18)
plt.ylabel("Property Type")
plt.xlabel("Percentage (%)")
plt.show()

In [ ]:
tup = (1, 5, 6), (4, 0, 2)

tup

In [ ]:
listed = [item for x in tup for item in x]

listed

In [ ]:
tup2 = tuple(["foo", [1, 2], True])

tup2

In [ ]:
# This will give you an error
# because there's no attribute of 'append'
tup2[0].append(3)

In [ ]:
# This will append the 3 to the list like [1, 2]; but it won't append in 'foo' or True
tup2[1].append(3)

In [ ]:
# 'tuple' object does not support item assignment
tup2[0] = False

In [ ]:
tup2

In [ ]:
a, b, c = tup2

In [ ]:
nested_tuple = (4, 2, 5), (8, 6)

nested_tuple

In [ ]:
# This is an example of not using a list comprehension
# You need to create a variable which has no values
empty_list = []

for x in nested_tuple:
    for item in x:
        empty_list.append(item)

print(empty_list)

In [ ]:
# This is an example of how to use a list comprehension
listed2 = [item for x in nested_tuple for item in x]

listed2